# Regression Week 5: LASSO (coordinate descent)

In this notebook, you will implement your very own LASSO solver via coordinate descent. You will:
* Write a function to normalize features
* Implement coordinate descent for LASSO
* Explore effects of L1 penalty

### Load libraries

In [242]:
import numpy as np
import pandas as pd

### Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [243]:
# load the data
dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 'sqft_living15':float, 'grade':int,
              'yr_renovated':int, 'price':float, 'bedrooms':float, 'zipcode':str, 'long':float, 'sqft_lot15':float,
              'sqft_living':float, 'floors':str, 'condition':int, 'lat':float, 'date':str, 'sqft_basement':int,
              'yr_built':int, 'id':str, 'sqft_lot':int, 'view':int}
sales = pd.read_csv('kc_house_data.csv', dtype=dtype_dict)
train_data = pd.read_csv('wk3_kc_house_train_data.csv', dtype=dtype_dict)
test_data = pd.read_csv('wk3_kc_house_test_data.csv', dtype=dtype_dict)
sales.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3.0,1.00,1180.0,5650,1,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340.0,5650.0
1,6414100192,20141209T000000,538000.0,3.0,2.25,2570.0,7242,2,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690.0,7639.0
2,5631500400,20150225T000000,180000.0,2.0,1.00,770.0,10000,1,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720.0,8062.0
3,2487200875,20141209T000000,604000.0,4.0,3.00,1960.0,5000,1,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360.0,5000.0
4,1954400510,20150218T000000,510000.0,3.0,2.00,1680.0,8080,1,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800.0,7503.0


In [244]:
# In the dataset, 'floors' was defined with type string, so we'll convert it to float and then to int,
# before using it below
for df in [sales, train_data, test_data]:
    df['floors'] = df['floors'].astype('float').astype(int)

If we want to do any "feature engineering" like creating new features or adjusting existing ones we should do this directly using the dataframes as seen in the first notebook of Week 2. For this notebook, however, we will work with the existing features.

### Import useful functions from previous notebook

As in Week 2, we convert the dataframe into a 2D Numpy array. Copy and paste `get_numpy_data()` from the second notebook of Week 2.

In [245]:
def get_numpy_data(dataframe, features, output):
    dataframe['constant'] = 1 # this is how you add a constant column to a dataframe
    # add the column 'constant' to the front of the features list so that we can extract it along with the others:
    features = ['constant'] + features # this is how you combine two lists
    # select the columns of the data given by the features list into the dataframe features_frame
    # (now including constant):
    features_frame = dataframe.loc[:, features]
    # the following line will convert the features dataframe into a numpy matrix:
    feature_matrix = features_frame.to_numpy()
    # assign the column of the data associated with the output to the array output_array
    output_array = dataframe['price']
    # the following will convert the array into a numpy array by first converting it to a list
    output_array = output_array.to_numpy()
    
    return(feature_matrix, output_array)

Also, copy and paste the `predict_output()` function to compute the predictions for an entire matrix of features given the matrix and the weights:

In [246]:
def predict_output(feature_matrix, weights):
    # assume feature_matrix is a numpy matrix containing the features as columns and weights is a corresponding
    # numpy array
    # create the predictions vector by using np.dot()
    predictions = np.dot(feature_matrix, weights)
    
    return(predictions)

### Normalize features
In the house dataset, features vary wildly in their relative magnitude: `sqft_living` is very large overall compared to `bedrooms`, for instance. As a result, weight for `sqft_living` would be much smaller than weight for `bedrooms`. This is problematic because "small" weights are dropped first as `l1_penalty` goes up. 

To give equal considerations for all features, we need to **normalize features** as discussed in the lectures: we divide each feature by its 2-norm so that the transformed feature has norm 1.

Let's see how we can do this normalization easily with Numpy: let us first consider a small matrix.

In [247]:
X = np.array([[3.,5.,8.], [4.,12.,15.]])
print(X)

[[ 3.  5.  8.]
 [ 4. 12. 15.]]


Numpy provides a shorthand for computing 2-norms of each column:

In [248]:
norms = np.linalg.norm(X, axis=0) # gives [norm(X[:,0]), norm(X[:,1]), norm(X[:,2])]
print(norms)

[ 5. 13. 17.]


To normalize, apply element-wise division:

In [249]:
print(X / norms) # gives [X[:,0]/norm(X[:,0]), X[:,1]/norm(X[:,1]), X[:,2]/norm(X[:,2])]

[[0.6        0.38461538 0.47058824]
 [0.8        0.92307692 0.88235294]]


Using the shorthand we just covered, write a short function called `normalize_features(feature_matrix)`, which normalizes columns of a given feature matrix. The function should return a pair `(normalized_features, norms)`, where the second item contains the norms of original features. As discussed in the lectures, we will use these norms to normalize the test data in the same way as we normalized the training data. 

In [250]:
def normalise_features(feature_matrix):
    X = np.array(feature_matrix)
    norms = np.linalg.norm(X, axis=0)
    normalised_features = X / norms
    
    return normalised_features, norms

To test the function, run the following:

In [251]:
normalised_features, norms = normalise_features(np.array([[3.,6.,9.], [4.,8.,12.]]))
print(normalised_features)
# should print
# [[ 0.6  0.6  0.6]
#  [ 0.8  0.8  0.8]]
print(norms)
# should print
# [5.  10.  15.]

[[0.6 0.6 0.6]
 [0.8 0.8 0.8]]
[ 5. 10. 15.]


### Implementing Coordinate Descent with normalized features

We seek to obtain a sparse set of weights by minimizing the LASSO cost function
```
SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|).
```
(By convention, we do not include `w[0]` in the L1 penalty term. We never want to push the intercept to zero.)

The absolute value sign makes the cost function non-differentiable, so simple gradient descent is not viable (you would need to implement a method called subgradient descent). Instead, we will use **coordinate descent**: at each iteration, we will fix all weights but weight `i` and find the value of weight `i` that minimizes the objective. That is, we look for
```
argmin_{w[i]} [ SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|) ]
```
where all weights other than `w[i]` are held to be constant. We will optimize one `w[i]` at a time, circling through the weights multiple times.  
  1. Pick a coordinate `i`
  2. Compute `w[i]` that minimizes the cost function `SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|)`
  3. Repeat Steps 1 and 2 for all coordinates, multiple times

For this notebook, we use **cyclical coordinate descent with normalized features**, where we cycle through coordinates 0 to (d-1) in order, and assume the features were normalized as discussed above. The formula for optimizing each coordinate is as follows:
```
       ┌ (ro[i] + lambda/2)     if ro[i] < -lambda/2
w[i] = ├ 0                      if -lambda/2 <= ro[i] <= lambda/2
       └ (ro[i] - lambda/2)     if ro[i] > lambda/2
```
where
```
ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ].
```

Note that we do not regularize the weight of the constant feature (intercept) `w[0]`, so, for this weight, the update is simply:
```
w[0] = ro[i]
```

### Effect of L1 penalty

Let us consider a simple model with 2 features:

In [252]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)

Don't forget to normalize features:

In [253]:
simple_feature_matrix, norms = normalise_features(simple_feature_matrix)

We assign some random set of initial weights and inspect the values of `ro[i]`:

In [254]:
weights = np.array([1., 4., 1.])

Use `predict_output()` to make predictions on this data.

In [255]:
prediction = predict_output(simple_feature_matrix, weights)
print(prediction)

[0.02675867 0.04339256 0.01990703 ... 0.02289873 0.03178473 0.02289873]


Compute the values of `ro[i]` for each feature in this simple model, using the formula:
```
ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ]
```

*Hint: You can get a Numpy vector for feature_i using:*
```
simple_feature_matrix[:,i]
```

In [256]:
num_features = np.shape(simple_feature_matrix)[1]
ro = np.zeros(num_features)

for feature in range(num_features):
    ro[feature] = np.sum(simple_feature_matrix[:, feature] * (
        output - prediction + weights[feature] * simple_feature_matrix[:, feature]))
print(ro)

[79400300.0145229  87939470.82325175 80966698.66623947]


***QUIZ QUESTION***

Recall that, whenever `ro[i]` falls between `-l1_penalty/2` and `l1_penalty/2`, the corresponding weight `w[i]` is sent to zero. Now suppose we were to take one step of coordinate descent on either feature 1 or feature 2. What range of values of `l1_penalty` **would not** set `w[1]` zero, but **would** set `w[2]` to zero, if we were to take a step in that coordinate? 

In [257]:
lambda_min_w1 = -(2 * ro[1])
lambda_max_w1 = 2 * ro[1]
# these values set w[1] to zero
print(lambda_min_w1)
print(lambda_max_w1)

-175878941.6465035
175878941.6465035


In [258]:
lambda_min_w2 = -(2 * ro[2])
lambda_max_w2 = 2 * ro[2]
# these values set w[2] to zero
print(lambda_min_w2)
print(lambda_max_w2)

-161933397.33247894
161933397.33247894


***QUIZ QUESTION***

What range of values of `l1_penalty` would set **both** `w[1]` and `w[2]` to zero, if we were to take a step in that coordinate? 

So we can say that `ro[i]` quantifies the significance of the i-th feature: the larger `ro[i]` is, the more likely it is for the i-th feature to be retained.

## Single Coordinate Descent Step

Using the formula above, implement coordinate descent that minimizes the cost function over a single feature i. Note that the intercept (weight 0) is not regularized. The function should accept feature matrix, output, current weights, l1 penalty, and index of feature to optimize over. The function should return new weight for feature i.

In [260]:
def lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty):
    # compute prediction
    prediction = predict_output(feature_matrix, weights)
    # compute ro[i] = SUM[ [feature_i]*(output - prediction + weight[i]*[feature_i]) ]
    ro_i = np.sum(feature_matrix[:, i] * (output - prediction + weights[i] * feature_matrix[:, i]))

    if i == 0: # intercept -- do not regularize
        new_weight_i = ro_i 
    elif ro_i < -l1_penalty/2.:
        new_weight_i = ro_i + (l1_penalty / 2.)
    elif ro_i > l1_penalty/2.:
        new_weight_i = ro_i - (l1_penalty / 2.)
    else:
        new_weight_i = 0.
    
    return new_weight_i

To test the function, run the following cell:

In [261]:
# should print 0.425558846691
import math
print(lasso_coordinate_descent_step(1, 
                                    np.array([[3./math.sqrt(13), 1./math.sqrt(10)], 
                                              [2./math.sqrt(13),3./math.sqrt(10)]]), 
                                    np.array([1., 1.]), 
                                    np.array([1., 4.]), 
                                    0.1))

0.4255588466910251


## Cyclical coordinate descent 

Now that we have a function that optimizes the cost function over a single coordinate, let us implement cyclical coordinate descent where we optimize coordinates 0, 1, ..., (d-1) in order and repeat.

When do we know to stop? Each time we scan all the coordinates (features) once, we measure the change in weight for each coordinate. If no coordinate changes by more than a specified threshold, we stop.

For each iteration:
1. As you loop over features in order and perform coordinate descent, measure how much each coordinate changes.
2. After the loop, if the maximum change across all coordinates is falls below the tolerance, stop. Otherwise, go back to step 1.

Return weights

**IMPORTANT: when computing a new weight for coordinate i, make sure to incorporate the new weights for coordinates 0, 1, ..., i-1. One good way is to update your weights variable in-place. See following pseudocode for illustration.**
```
for i in range(len(weights)):
    old_weights_i = weights[i] # remember old value of weight[i], as it will be overwritten
    # the following line uses new values for weight[0], weight[1], ..., weight[i-1]
    #     and old values for weight[i], ..., weight[d-1]
    weights[i] = lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty)
    
    # use old_weights_i to compute change in coordinate
    ...
```

In [262]:
def lasso_cyclical_coordinate_descent(feature_matrix, output, initial_weights, l1_penalty, tolerance):
    # initialise weights, old_weights and weight change arrays
    weights = initial_weights.copy()
    old_weights = weights.copy()
    weight_changes = np.zeros(len(weights))
    
    # set the while condition
    continue_updating = True
    
    itr = 1
    while continue_updating:

        for i in range(len(weights)):
            old_weights[i] = weights[i]
            weights[i] = lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty)
            # compute the change in coordinate value
            weight_changes[i] = np.abs(old_weights[i] - weights[i])
        
        # compare the change to the specified tolerance
        if np.max(weight_changes) < tolerance:
            continue_updating = False
            print("Maximum change across all coordinates is less than the specified tolerance.")
            print("Loop stopped at iteration: " + str(itr))
            print("New weights: " + str(weights))
            
        # increment itr by one each time
        itr +=1

    return weights

Using the following parameters, learn the weights on the sales dataset. 

In [263]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
initial_weights = np.zeros(3)
l1_penalty = 1e7
tolerance = 1.0

First create a normalized version of the feature matrix, `normalized_simple_feature_matrix`.

In [264]:
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)
(normalised_simple_feature_matrix, simple_norms) = normalise_features(simple_feature_matrix) # normalize features

Then, run your implementation of LASSO coordinate descent:

In [265]:
weights = lasso_cyclical_coordinate_descent(normalised_simple_feature_matrix, output, initial_weights, l1_penalty, tolerance)

Maximum change across all coordinates is less than the specified tolerance.
Loop stopped at iteration: 93
New weights: [21624997.9595191  63157247.20788956        0.        ]


***QUIZ QUESTIONS***
1. What is the RSS of the learned model on the normalized dataset? (Hint: use the normalized feature matrix when you make predictions.)
2. Which features had weight zero at convergence?

In [266]:
predictions = predict_output(normalised_simple_feature_matrix, weights)
RSS = np.sum((sales['price'] - predictions)**2)
RSS

1630492476715386.5

Feature 2 i.e. 'bedrooms' had weight zero at convergence. (The first weight in the array corresponds to the constant, and the second to 'sqft_living')

### Evaluating LASSO fit with more features

Let us split the sales dataset into training and test sets.

In [267]:
train_data.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,2487200875,20141209T000000,604000.0,4.0,3.00,1960.0,5000,1,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360.0,5000.0
1,7237550310,20140512T000000,1225000.0,4.0,4.50,5420.0,101930,1,0,0,...,11,3890,1530,2001,0,98053,47.6561,-122.005,4760.0,101930.0
2,9212900260,20140527T000000,468000.0,2.0,1.00,1160.0,6000,1,0,0,...,7,860,300,1942,0,98115,47.6900,-122.292,1330.0,6000.0
3,0114101516,20140528T000000,310000.0,3.0,1.00,1430.0,19901,1,0,0,...,7,1430,0,1927,0,98028,47.7558,-122.229,1780.0,12697.0
4,6054650070,20141007T000000,400000.0,3.0,1.75,1370.0,9680,1,0,0,...,7,1370,0,1977,0,98074,47.6127,-122.045,1370.0,10208.0


In [268]:
test_data.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,3793500160,20150312T000000,323000.0,3.0,2.50,1890.0,6560,2,0,0,...,7,1890,0,2003,0,98038,47.3684,-122.031,2390.0,7570.0
1,1175000570,20150312T000000,530000.0,5.0,2.00,1810.0,4850,1,0,0,...,7,1810,0,1900,0,98107,47.6700,-122.394,1360.0,4850.0
2,0016000397,20141205T000000,189000.0,2.0,1.00,1200.0,9850,1,0,0,...,7,1200,0,1921,0,98002,47.3089,-122.210,1060.0,5095.0
3,0461000390,20140624T000000,687500.0,4.0,1.75,2330.0,5000,1,0,0,...,7,1510,820,1929,0,98117,47.6823,-122.368,1460.0,5000.0
4,7895500070,20150213T000000,240000.0,4.0,1.00,1220.0,8075,1,0,0,...,7,890,330,1969,0,98001,47.3341,-122.282,1290.0,7800.0


Let us consider the following set of features.

In [391]:
all_features = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
                'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated']

First, create a normalized feature matrix from the TRAINING data with these features.  (Make you store the norms for the normalization, since we'll use them later)

In [392]:
output = 'price'

(feature_matrix, output) = get_numpy_data(train_data, all_features, output)
(normalised_feature_matrix, norms) = normalise_features(feature_matrix) # normalize features

First, learn the weights with `l1_penalty=1e7`, on the training data. Initialize weights to all zeros, and set the `tolerance=1`.  Call resulting weights `weights1e7`, you will need them later.

In [393]:
initial_weights = np.zeros(len(all_features)+1)
l1_penalty = 1e7
tolerance = 1.0

weights1e7 = lasso_cyclical_coordinate_descent(normalised_feature_matrix, output, initial_weights, l1_penalty, tolerance)

Maximum change across all coordinates is less than the specified tolerance.
Loop stopped at iteration: 91
New weights: [23864692.5095384         0.                0.         30495548.13254719
        0.                0.          1901633.61475593  5705765.01673266
        0.                0.                0.                0.
        0.                0.        ]


***QUIZ QUESTION***

What features had non-zero weight in this case?

In [394]:
weights_df = pd.DataFrame(['constant'] + all_features, weights1e7).reset_index()
weights_df.columns = ['weight', 'feature']
weights_df.loc[weights_df.weight > 0]

,weight,feature
0,2.386469e+07,constant
3,3.049555e+07,sqft_living
6,1.901634e+06,waterfront
7,5.705765e+06,view


Next, learn the weights with `l1_penalty=1e8`, on the training data. Initialize weights to all zeros, and set the `tolerance=1`.  Call resulting weights `weights1e8`, you will need them later.

In [395]:
initial_weights = np.zeros(len(all_features)+1)
l1_penalty = 1e8
tolerance = 1.0

weights1e8 = lasso_cyclical_coordinate_descent(normalised_feature_matrix, output, initial_weights, l1_penalty, tolerance)

Maximum change across all coordinates is less than the specified tolerance.
Loop stopped at iteration: 2
New weights: [53621004.68971471        0.                0.                0.
        0.                0.                0.                0.
        0.                0.                0.                0.
        0.                0.        ]


***QUIZ QUESTION***

What features had non-zero weight in this case?

In [396]:
weights_df = pd.DataFrame(['constant'] + all_features, weights1e8).reset_index()
weights_df.columns = ['weight', 'feature']
weights_df.loc[weights_df.weight > 0]

,weight,feature
0,5.362100e+07,constant


Finally, learn the weights with `l1_penalty=1e4`, on the training data. Initialize weights to all zeros, and set the `tolerance=5e5`.  Call resulting weights `weights1e4`, you will need them later.  (This case will take quite a bit longer to converge than the others above.)

In [397]:
initial_weights = np.zeros(len(all_features)+1)
l1_penalty = 1e4
tolerance = 5e5

weights1e4 = lasso_cyclical_coordinate_descent(normalised_feature_matrix, output, initial_weights, l1_penalty, tolerance)

Maximum change across all coordinates is less than the specified tolerance.
Loop stopped at iteration: 62
New weights: [ 57039550.90409693 -14150972.67079931  14513408.78811389
  57113668.1447304   -1594108.67091326  -8545760.68720604
   5346859.58126979   5804365.12288566   -751534.05515794
   6121506.76396564    739422.64716264  -1722168.20964147
 -54864499.949995     2127572.62934027]


***QUIZ QUESTION***

What features had non-zero weight in this case?

In [398]:
weights_df = pd.DataFrame(['constant'] + all_features, weights1e4).reset_index()
weights_df.columns = ['weight', 'feature']
weights_df.loc[weights_df.weight > 0]

,weight,feature
0,5.703955e+07,constant
2,1.451341e+07,bathrooms
3,5.711367e+07,sqft_living
6,5.346860e+06,waterfront
7,5.804365e+06,view
9,6.121507e+06,grade
10,7.394226e+05,sqft_above
13,2.127573e+06,yr_renovated


## Rescaling learned weights

Recall that we normalized our feature matrix, before learning the weights.  To use these weights on a test set, we must normalize the test data in the same way.

Alternatively, we can rescale the learned weights to include the normalization, so we never have to worry about normalizing the test data: 

In this case, we must scale the resulting weights so that we can make predictions with *original* features:
 1. Store the norms of the original features to a vector called `norms`:
```
features, norms = normalize_features(features)
```
 2. Run Lasso on the normalized features and obtain a `weights` vector
 3. Compute the weights for the original features by performing element-wise division, i.e.
```
weights_normalized = weights / norms
```
Now, we can apply `weights_normalized` to the test data, without normalizing it!

Create a normalized version of each of the weights learned above. (`weights1e4`, `weights1e7`, `weights1e8`).

In [403]:
normalised_weights1e4 = weights1e4 / norms
print(normalised_weights1e4[3])

253.16569712338605


In [404]:
normalised_weights1e7 = weights1e7 / norms
print(normalised_weights1e7[3])

135.17651646138245


In [405]:
normalised_weights1e8 = weights1e8 / norms
print(normalised_weights1e8[3])

0.0


To check your results, if you call `normalized_weights1e7` the normalized version of `weights1e7`, then:
```
print normalized_weights1e7[3]
```
should return 161.31745624837794.

## Evaluating each of the learned models on the test data

Let's now evaluate the three models on the test data:

In [406]:
(test_feature_matrix, test_output) = get_numpy_data(test_data, all_features, 'price')

Compute the RSS of each of the three normalized weights on the (unnormalized) `test_feature_matrix`:

In [411]:
predictions1e4 = predict_output(test_feature_matrix, normalised_weights1e4)
RSS1e4 = np.sum((test['price'] - predictions1e4)**2)
RSS1e4

128218610476653.95

In [412]:
predictions1e7 = predict_output(test_feature_matrix, normalised_weights1e7)
RSS1e7 = np.sum((test['price'] - predictions1e7)**2)
RSS1e7

163103564164999.12

In [413]:
predictions1e8 = predict_output(test_feature_matrix, normalised_weights1e8)
RSS1e8 = np.sum((test['price'] - predictions1e8)**2)
RSS1e8

284718925209874.0

***QUIZ QUESTION***

Which model performed best on the test data?

In [414]:
np.min([RSS1e4, RSS1e7, RSS1e8])

128218610476653.95